# Chapter 10: Introduction to Artificial Neural Networks with Keras

**Artificial neural networks (ANNs)**: A Machine Learning model inspired by the networks of biological neurons found in our brains.

## 10.1 From Biological to Artificial Neurons

### 10.1.1 Biological Neurons

Cell body -> Axon -> Telondendria -> Synaptic terminals (synapses) => Next cell's dendrites/body

> Note: See Figure 10-1 in book

Biological neurons produce short electrical impulses called action potentials (APs, or just signals) which travel along the axons and make the synapses release chemical signals called neurotransmitters.

When a neuron receives a sufficient amount of these neurotransmitters, it fires its own electrical impulses

### 10.1.2 Logical Computations with Neurons

Artificial neuron - It has one or more binary (on/off) inputs and one binary output. The artificial neuron activates its output when more than a certain number of its inputs are active.

> Note: See Figure 10-3 in book.

> Note: 2 input signals are needed to activate neuron C.

1. Identity function: if neuron A is activated, neuron C gets activated as well. But if neuron A is off, then neuron C is off as well.

2. Logical AND: neuron C is activated only when both neurons A and B are activated (a single input signal is not enough to activate neuron C).

3. Logical OR: neuron C gets activated if either neuron A or B is activated (or both).

4. Logicial NOT: neuron C is activated only if neuron A is active and neuron B is off.

### 10.1.3 The Perceptron

### 10.1.4 The Multilayer Perceptron and Backpropagation

### 10.1.5 Regression MLPs

### 10.1.6 Classification MLPs

## 10.2 Implementing MLPs with Keras

### 10.2.1 Installing TensorFlow 2

### 10.2.2 Building an Image Classifier Using the Sequential API

### 10.2.3 Building a Regression MLP Using the Sequential API

### 10.2.4 Building Complex Models Using the Functional API

### 10.2.5 Using the Subclassing API to Build Dynamic Models

### 10.2.6 Saving and Restoring a Model

### 10.2.7 Using Callbacks

### 10.2.8 Using TensorBoard for Visualization

## 10.3 Fine-Tuning Neural Networks Hyperparameters

### 10.3.1 Number of Hidden Layers

### 10.3.2 Number of Neurons per Hidden Layer

### 10.3.3 Learning Rate, Batch Size, and Other Hyperparameters